In [ ]:
# cnn_model.ipynb

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalMaxPooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.utils.class_weight import compute_sample_weight

In [ ]:
# === Config ===
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

In [ ]:
# === Load CSVs ===
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep_Learning/MURA-v1.1/train_image_metadata.csv')
df_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep_Learning/MURA-v1.1/valid_image_metadata.csv')

In [ ]:
# === Image loading and preprocessing ===
def load_and_preprocess(path):
    img = cv2.imread(path.decode('utf-8'), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = img.astype('float32') / 255.0
    return img

In [ ]:
def tf_preprocess(path, label):
    img = tf.numpy_function(load_and_preprocess, [path], tf.float32)
    img.set_shape((IMAGE_SIZE, IMAGE_SIZE, 3))
    return img, label

In [ ]:

# === tf.data pipeline ===
def build_dataset(df, training=True):
    paths = df['image_path'].values
    labels = df['label'].values.astype(np.int32)
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    dataset = dataset.map(tf_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        dataset = dataset.shuffle(1000)
    return dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# === Add sample weights to the dataset ===
sample_weights = compute_sample_weight('balanced', df_train['label'].values)
train_ds = tf.data.Dataset.from_tensor_slices((df_train['image_path'].values, df_train['label'].values, sample_weights))

def tf_preprocess_with_weights(path, label, weight):
    img = tf.numpy_function(load_and_preprocess, [path], tf.float32)
    img.set_shape((IMAGE_SIZE, IMAGE_SIZE, 3))
    return img, label, weight

train_ds = train_ds.map(tf_preprocess_with_weights, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = build_dataset(df_val, training=False)

In [ ]:
# === Compute sample weights for training ===
# sample_weights = compute_sample_weight('balanced', df_train['label'].values)


In [ ]:
# === Model definition ===
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model.trainable = False  # freeze base initially

inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalMaxPooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
# === Callbacks ===
callbacks = [
    EarlyStopping(patience=4, restore_best_weights=True, monitor='val_loss'),
    ReduceLROnPlateau(patience=2, factor=0.2, monitor='val_loss')
]

In [12]:
# === Train ===
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1,
)


Epoch 1/20
 607/1151 ━━━━━━━━━━━━━━━━━━━━ 42:05 5s/step - accuracy: 0.8177 - loss: 0.4625 - precision: 0.8233 - recall: 0.8659

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  Error in user-defined function passed to ParallelMapDatasetV2:1 transformation with iterator: Iterator::Root::Prefetch::BatchV2::Shuffle::ParallelMapV2: error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "<ipython-input-5-33ba07433d2a>", line 4, in load_and_preprocess
    img = cv2.resiz

In [ ]:
# === Save model ===
model.save('resnet_mura_binary.h5')

In [ ]:
# Evaluate using metrics.py module
y_true, y_pred = [], []

for batch_x, y_batch in val_ds:
    preds = model.predict(batch_x, verbose=0)
    y_true.extend(y_batch.numpy().flatten())
    y_pred.extend(preds.numpy().flatten())

import numpy as np
from metrics import get_binary_metrics

metrics = get_binary_metrics(np.array(y_true), np.array(y_pred))
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")


In [ ]:
# Plot training curves
import matplotlib.pyplot as plt

def plot_history(history):
    plt.figure(figsize=(12, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title("Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
plot_history(history)


In [ ]:
# === Save model ===
model.save('resnet_mura_binary.h5')